### Pytorch QUICKSTART
##### LINK: https://pytorch.org/tutorials/beginner/basics/quickstart_tutorial.html

In [2]:
import torch
from torch import nn
from torch.utils.data import DataLoader
from torchvision import datasets
from torchvision.transforms import ToTensor

In [6]:
training_data = datasets.FashionMNIST(
    root="../../datasets/pytorch_tutorials",
    train=True,
    download=True,
    transform=ToTensor(),
)

test_data = datasets.FashionMNIST(
    root="../../datasets/pytorch_tutorials",
    train=False,
    download=True,
    transform=ToTensor(),
)

  0%|          | 0/26421880 [00:00<?, ?it/s]

Extracting ../../datasets/pytorch_tutorials/FashionMNIST/raw/train-images-idx3-ubyte.gz to ../../datasets/pytorch_tutorials/FashionMNIST/raw



  0%|          | 0/29515 [00:00<?, ?it/s]

Extracting ../../datasets/pytorch_tutorials/FashionMNIST/raw/train-labels-idx1-ubyte.gz to ../../datasets/pytorch_tutorials/FashionMNIST/raw



  0%|          | 0/4422102 [00:00<?, ?it/s]

Extracting ../../datasets/pytorch_tutorials/FashionMNIST/raw/t10k-images-idx3-ubyte.gz to ../../datasets/pytorch_tutorials/FashionMNIST/raw



  0%|          | 0/5148 [00:00<?, ?it/s]

Extracting ../../datasets/pytorch_tutorials/FashionMNIST/raw/t10k-labels-idx1-ubyte.gz to ../../datasets/pytorch_tutorials/FashionMNIST/raw



In [8]:
batch_size = 64

# Create data loaders
train_dataloader = DataLoader(training_data, batch_size=batch_size)
test_dataloader = DataLoader(test_data, batch_size=batch_size)

for X, y in test_dataloader:
    print(f"Shape of X [N, C, H, W]: {X.shape}")
    print(f"Shape of y: {y.shape} {y.dtype}")
    break

Shape of X [N, C, H, W]: torch.Size([64, 1, 28, 28])
Shape of y: torch.Size([64]) torch.int64


In [12]:
### CREATING MODELS ###
device = "cuda" if torch.cuda.is_available() else "cpu"

print(f"Using {device} device")

# Define model
class NeuralNetwork(nn.Module):
    def __init__(self):
        super(NeuralNetwork, self).__init__()
        self.flatten = nn.Flatten()
        self.linear_relu_stack = nn.Sequential(
            nn.Linear(28*28, 512),
            nn.ReLU(),
            nn.Linear(512, 512),
            nn.ReLU(),
            nn.Linear(512, 10)
        )
        
    def forward(self, x):
        x = self.flatten(x)
        logits = self.linear_relu_stack(x)
        return logits

model = NeuralNetwork().to(device)
print(model)

Using cuda device
NeuralNetwork(
  (flatten): Flatten(start_dim=1, end_dim=-1)
  (linear_relu_stack): Sequential(
    (0): Linear(in_features=784, out_features=512, bias=True)
    (1): ReLU()
    (2): Linear(in_features=512, out_features=512, bias=True)
    (3): ReLU()
    (4): Linear(in_features=512, out_features=10, bias=True)
  )
)


In [13]:
loss_fn = nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(model.parameters(), lr=1e-3)

In [17]:
def train(dataloader, model, loss_fn, optimizer):
    size = len(dataloader.dataset)
    model.train()
    for batch, (X, y) in enumerate(dataloader):
        X = X.to(device)
        y = y.to(device)
        
        # Compute prediction error
        pred = model(X)
        loss = loss_fn(pred, y)
        
        # Backpropagation
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        
        if batch % 100 == 0:
            loss = loss.item()
            current = batch * len(X)
            print(f"loss: {loss:>7f} [{current:>5d}]/{size:>5d}]")
            
def test(dataloader, model, loss_fn):
    size = len(dataloader.dataset)
    num_batches = len(dataloader)
    model.eval()
    test_loss, correct = 0, 0
    with torch.no_grad():
        for X, y in dataloader:
            X = X.to(device)
            y = y.to(device)
            pred = model(X)
            test_loss += loss_fn(pred, y).item()
            correct += (pred.argmax(1) == y).type(torch.float).sum().item()
    
    test_loss /= num_batches
    correct /= size
    print(f"Test Error: \n Accuracy: {(100*correct):>0.1f}%, Avg loss: {test_loss:>8f} \n")


In [19]:
epochs = 15

for t in range(epochs):
    print(f"Epoch {t+1}\n-------------------------------")
    train(train_dataloader, model, loss_fn, optimizer)
    test(test_dataloader, model, loss_fn)

print("Done!")

Epoch 1
-------------------------------
loss: 1.039816 [    0]/60000]
loss: 1.052227 [ 6400]/60000]
loss: 0.867296 [12800]/60000]
loss: 1.019560 [19200]/60000]
loss: 0.903219 [25600]/60000]
loss: 0.922184 [32000]/60000]
loss: 0.968943 [38400]/60000]
loss: 0.930818 [44800]/60000]
loss: 0.953914 [51200]/60000]
loss: 0.891871 [57600]/60000]
Test Error: 
 Accuracy: 67.2%, Avg loss: 0.907162 

Epoch 2
-------------------------------
loss: 0.953414 [    0]/60000]
loss: 0.983684 [ 6400]/60000]
loss: 0.785557 [12800]/60000]
loss: 0.954970 [19200]/60000]
loss: 0.845227 [25600]/60000]
loss: 0.855641 [32000]/60000]
loss: 0.913971 [38400]/60000]
loss: 0.880806 [44800]/60000]
loss: 0.896548 [51200]/60000]
loss: 0.844030 [57600]/60000]
Test Error: 
 Accuracy: 68.1%, Avg loss: 0.855851 

Epoch 3
-------------------------------
loss: 0.887204 [    0]/60000]
loss: 0.933089 [ 6400]/60000]
loss: 0.724738 [12800]/60000]
loss: 0.907117 [19200]/60000]
loss: 0.804354 [25600]/60000]
loss: 0.806343 [32000]/600

In [20]:
torch.save(model.state_dict(), "test_model.pth")
print("Saved PyTorch Model State to test_model.pth")

Saved PyTorch Model State to test_model.pth


In [22]:
model = NeuralNetwork()
model.load_state_dict(torch.load("test_model.pth"))

<All keys matched successfully>

In [ ]:
### PREDICTIONS ###

In [24]:
classes = [
    "T-shirt/top",
    "Trouser",
    "Pullover",
    "Dress",
    "Coat",
    "Sandal",
    "Shirt",
    "Sneaker",
    "Bag",
    "Ankle boot",
]

In [28]:
model.eval()
x, y = test_data[0][0], test_data[0][1]
with torch.no_grad():
    pred = model(x)
    predicted, actual = classes[pred[0].argmax(0)], classes[y]
    print(f'Predicted: "{predicted}", Actual: "{actual}"')

Predicted: "Ankle boot", Actual: "Ankle boot"
